# Comparing Canadian Living Wages - Project Proposal

### STAT 201 2021W2 - Group 5

By Michael Guo, Joshua Liu, Samuel Leung

## Introduction

Many parents envision a future where their child can flourish and live independently from them. While independence is challenging to quantify, an important predictor of (general) independence is financial independence. The literature describes many contributing factors to financial independence, including income, education, communication skills, and cost of living (Xiao, Chatterjee, & Kim, 2014; Lee & Mortimer, 2009). 

This analysis seeks to answer the question, "Has there been a significant change the opportunity a minimum-wage worker in Canada has for financial independence between the years 1990 and 2021?". The minimum wage (numerical variable `min_wage`) represents the income factor of financial independence while the average cost of rent of a one-bedroom apartment (numerical variable `rent`) represents the cost of living. 

The data is obtained from two curated online datasets, which themselves obtained their data from two separate datasets published by Statistics Canada (see References). The first Statistics Canada dataset describes the average rent in Canadian cities with population greater than 10000. The second Statistics Canada dataset records the minimum wage in each province. These two datasets were merged, tidied, and wrangled by the online source to provide two datasets with the average rent of a one-bedroom apartment and minimum wage for the years 1990 and 2021. 

The response variable is the ratio between average weekly rent and weekly income (at minimum wage) in a city. This variable is labeled `living_ratio`. A location parameter that helps answer our question is the mean living ratio across all cities in Canada with a population greater than 10000. Estimating the mean across its major cities will provide representative insight into the living conditions of the population of Canadian minimum-wage workers. A scale parameter that helps provide additional insight is the standard deviation of the living ratio. It would indicate whether the living conditions in Canada are homogeneous or whether some cities are more favourable to minimum-wage workers than others. 

## Preliminary Results



Loading Required Packages:

In [ ]:
install.packages("infer")
install.packages("tidyverse")
library(cowplot)
# library(datateachr)
# library(digest)
library(infer)
# library(repr)
# library(taxyvr)
library(tidyverse)

Reading in Data from the Web:

In [ ]:
rentwage1990 <- read.csv('https://raw.githubusercontent.com/Guo-Michael/stat_201_project/main/rentwage1990.csv')
rentwage2021 <- read.csv('https://raw.githubusercontent.com/Guo-Michael/stat_201_project/main/rentwage2021.csv')

Cleaning and Wrangling Data:

In [ ]:
# Selecting Relevant Features from the Datasets
rentwage1990filtered <- rentwage1990 %>%
    select(year, city, province, min_wage, rent)
rentwage2021filtered <- rentwage2021 %>%
    select(year, city, province, min_wage, rent)

# Merging Both Datasets
rentwage_bothyears <- bind_rows(rentwage1990filtered, rentwage2021filtered)

# Checking for Data Inconsistency Over the Years
# If data for a city only appears for one year and not the other,
# the data is incomplete, and therefore will need to be removed
# before analysis.

# Checking for cities that appear in both years
double_cities <- rentwage_bothyears %>%
    group_by(city) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    filter(n == 2) %>%
    pull(city)

# Filtering away cities that do not appear in both years
rentwage_clean <- rentwage_bothyears %>%
    filter(city %in% double_cities) %>%
    select(-province) %>%
# Constructing Living Ratio for both years, switching units to $/week
    mutate(living_ratio = (rent * 12 / 52) / (min_wage * 35)) %>%
    select(-min_wage, -rent) %>%
# Wrangling Data to show years as data features, not as a variable
    mutate(year = ifelse(year == 1990, "year1990", "year2021")) %>%
    pivot_wider(names_from = year, values_from = living_ratio)

# test_stat <- rentwage_2021_clean %>%
#     mutate(diff_in_ratio = year2021 - year1990) %>%
#     summarize(test_stat = mean(diff_in_ratio)) %>%
#     pull()
# test_stat

Exploratory Visualization:

In [ ]:
# Visualizaing 2021 Living Ratio via Histogram
sample_dist_2021 <- rentwage_clean %>%
    ggplot(aes(x = year2021)) +
    geom_histogram(binwidth = 0.03) +
    ggtitle("2021 Sample Distribution") +
    xlab("Living Ratio") +
    xlim(0, 0.6) +
    ylim(0, 30) 

# Visualizaing 1990 Living Ratio via Histogram
sample_dist_1990 <- rentwage_clean %>%
    ggplot(aes(x = year1990)) +
    geom_histogram(binwidth = 0.03) +
    ggtitle("1990 Sample Distribution") +
    xlab("Living Ratio") +
    xlim(0, 0.6) +
    ylim(0, 30)

# Displaying Plots Side by Side
plots <- plot_grid(sample_dist_1990, sample_dist_2021)
plots

Computing Parameter Estimates: Living Ratio Means from 1990 and 2021

In [ ]:
living_ratio_estimates <- rentwage_clean %>%
    summarize(ratio_mean1990 = mean(year1990),
              ratio_mean2021 = mean(year2021))
living_ratio_estimates

## Methods: Plan

The ratio of average daily rent for a one-bedroom apartment to daily minimum wage for major Canadian cities, called the living ratio, will be used to quantify the cost of financial independence. We believe this quantification is reasonable and trustworthy, as being able to afford personal housing is a common measure for independence, and minimum wage is a way to quantify earning power. Rent and minimum wage data is taken from reports from the Government of Canada, which also demonstrates data reliability.

The histograms generated provide preliminary insight into the differences in living ratios across Canadian cities, but do not provide a conclusive argument on how the ratios changed across the 31 years. Furthermore, without deeper inferential analysis, the significance of the differences across the ratio estimates is inconclusive.

Hypothesis testing will be conducted on this difference quantity, with the null hypothesis being that there is no difference between the 1990 and 2021 living ratios. Testing will be conducted through bootstrapping and asymptotics methods, via resampling the data many times as well as using the Central Limit Theorem to approximate the null distribution. Since the data points from 1990 and 2021 are collected from the same Canadian cities, these two samples are not independent, and therefore a two-sample t-test cannot be used. Instead, a paired one-sample t-test will be used, with each city being one observation, and the test statistic being the difference of living ratios across 2021 and 1990. Confidence intervals will be similarly conducted using the previously mentioned methods to assess the significance in ratio differences.

We expect to find that the cost of financial independence has changed since 1990, due to the many historical, political, and economic progressions in the 31-year gap. Such findings could inspire deeper economic analysis to better understand and support future generations, providing more tailored resources to prepare for economic and individual stability, whether through the government or through the community. Future analysis can include other variables influencing financial independence, or modelling and analyzing a time series that includes data for each year, providing more insight to the changes in the past decades.    

## References

!!!Josh you need two (scientific) references for your intro

!!!ALSO we need consistent format (APA)

Lee, J. C., & Mortimer, J. T. (2009). Family Socialization, Economic Self-Efficacy, and the Attainment of Financial Independence in Early Adulthood. *Longitudinal and life course studies : international journal, 1*(1), 45–62.

Xiao, J. J., Chatterjee, S., &amp; Kim, J. (2014). Factors associated with financial independence of Young Adults. *International Journal of Consumer Studies, 38*(4), 394–403. https://doi.org/10.1111/ijcs.12106 

Rent Data: https://open.canada.ca/data/en/dataset/1146388b-a150-4e70-98ec-eb40cb9083c8/resource/2be23210-9dd2-4511-840e-c467e0645827

Minimum Wages: http://srv116.services.gc.ca/dimt-wid/sm-mw/rpt2.aspx

hi I am joshua (hello I am Michael)
